##### Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import re
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

##### Navegador

In [2]:
nav = webdriver.Chrome()

##### Base de dados

In [3]:
tabela= pd.read_excel('buscas.xlsx')
display(tabela)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


##### Definindo Funções para o Google Shopping e Buscapé

In [4]:
def verificar_tem_termos_banidos(lista_termos_banidos, nome):
  tem_termos_banidos = False
  for palavra in termos_banidos.split(' '):
    if palavra in nome:
      tem_termos_banidos = True
  return tem_termos_banidos

def verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome):
    tem_todos_termos_produtos = True
    for palavra in lista_termos_nome_produto:
      if palavra not in nome:
        tem_todos_termos_produtos = False   
    return tem_todos_termos_produtos

def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.google.com.br/')

  nav.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER)

  # Aba Shopping
  elementos = nav.find_element(By.CLASS_NAME, 'UqcIvb').click()

  # Preços
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'KZmu8e')

  for resultado in lista_resultados:
    nome = resultado.find_element(By.CLASS_NAME, 'sh-np__product-title').text
    nome = nome.lower()

    tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
    tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)

    if not tem_termos_banidos and tem_todos_termos_produtos:    
      preco = resultado.find_element(By.CLASS_NAME, 'hn9kf').text
      preco = re.search(r'\d+\.?\d+', preco).group()
      preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
      preco = float(preco)

      if preco_minimo <= preco <= preco_maximo:
        elemento_referencia = resultado.find_element(By.CLASS_NAME, 'SirUVb')
        elemento_pai = elemento_referencia.find_element(By.XPATH, './..')
        link = elemento_pai.get_attribute('href')
        lista_ofertas.append((nome, preco, link))
  return lista_ofertas


def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):

  produto = produto.lower()
  termos_banidos = termos_banidos.lower()
  lista_termos_banidos = termos_banidos.split(' ')
  lista_termos_nome_produto = produto.split(' ')
  lista_ofertas = []
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  nav.get('https://www.buscape.com.br/')
  nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)

  time.sleep(5)
    
  lista_resultados = nav.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')

  for resultado in lista_resultados:
    preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text
    nome = resultado.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text
    nome = nome.lower()
    link = resultado.get_attribute('href')

    tem_termos_banidos = verificar_tem_termos_banidos(lista_termos_banidos, nome)
    tem_todos_termos_produtos = verificar_tem_todos_termos_produtos(lista_termos_nome_produto, nome)

    if not tem_termos_banidos and tem_todos_termos_produtos:  
      preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
      preco = float(preco)
      if preco_minimo <= preco <= preco_maximo:
        lista_ofertas.append((nome, preco, link))

  return lista_ofertas

##### Gerando Resultados em DataFrames (Lista Ofertas)

In [5]:
tabela_ofertas =  pd.DataFrame()

for linha in tabela.index:
  produto = tabela.loc[linha, 'Nome']
  termos_banidos = tabela.loc[linha, 'Termos banidos']
  preco_minimo = tabela.loc[linha, 'Preço mínimo']
  preco_maximo = tabela.loc[linha, 'Preço máximo']

  lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
  if lista_ofertas_google_shopping:
    tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
    tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping])
  else:
    tabela_google_shopping = None

  lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
  if lista_ofertas_buscape:
    tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
    tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape])
  else:
    tabela_buscape = None

display(tabela_ofertas)    

,produto,preco,link
0,"smartphone apple iphone 12, 64gb, branco, 5g, ...",3499.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
1,"iphone 12 64gb roxo tela 6,1 4g câmera traseir...",3349.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
2,vitrine iphone 12 preto 64gb,3199.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
3,"iphone 12 64gb preto de vitrine tela 6,1 4g câ...",3349.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
4,"iphone 12 64gb branco de vitrine tela 6,1&quot...",3349.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
5,vitrine iphone 12 verde 64gb,3199.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
6,celular apple iphone 12 black 64gb vitrine/sem...,3379.00,https://www.google.com.br/aclk?sa=l&ai=DChcSEw...
0,smartphone apple iphone 12 64gb câmera dupla,3023.00,https://www.buscape.com.br/celular/smartphone-...
0,placa de video nvidia geforce rtx 3060 ti 8 gb...,4108.27,https://www.buscape.com.br/placa-de-video/plac...


##### Exportando para Excel

In [6]:
tabela_ofertas.to_excel('Ofertas.xlsx', index=False)

##### Enviado e-mail

In [7]:
if len(tabela_ofertas) > 0:

  from_addr = 'gsoaresdesouzaaws@gmail.com'
  to_addr = 'gsoaresdesouzaaws@gmail.com'
  password = 'gtnthbnygcqsmufm'

  msg = MIMEMultipart()
  msg['From'] = from_addr
  msg['To'] = to_addr
  msg['Subject'] = 'Produto(s) Encontrado(s) na faixa de preço desejada'

  body = f''''
  <p>Prezados,</p>
  <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada.</p>
  {tabela_ofertas.to_html(index=False)}
  <p>Atenciosamente,</p>
  '''
  msg.attach(MIMEText(body, 'html'))

  server = smtplib.SMTP('smtp.gmail.com', 587)
  server.starttls()

  server.login(from_addr, password)
  text = msg.as_string()
  server.sendmail(from_addr, to_addr, text)
  server.quit()
  nav.quit()